[View in Colaboratory](https://colab.research.google.com/github/jayaramanjay97/Conv2D/blob/inte/First.ipynb)

In [0]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def conv_single_step(a_slice_prev, W, b):
    s =np.multiply(a_slice_prev,W)
    Z = np.sum(s)
    Z = Z+float(b)
    return Z

In [0]:
def conv_forward(A_prev, W, b, hparameters):
    (m,n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    (f, f, n_C_prev, n_C) = W.shape

    stride = hparameters["stride"]
    pad = hparameters["pad"]

    
    n_H = int((n_H_prev - f + 2*pad) / stride )+ 1
    n_W = int((n_W_prev - f + 2*pad) / stride )+ 1

       
    Z = np.zeros((m,n_H, n_W, n_C))
    
    for i in range(m):
      a_prev = A_prev[i,:,:,:]  
      for h in range(n_H):                         
        for w in range(n_W):                       
          for c in range(n_C):                   

          
            vert_start = stride * h
            vert_end = vert_start + f
            horiz_start = stride * w
            horiz_end = horiz_start + f

            a_slice_prev = a_prev[vert_start:vert_end, horiz_start:horiz_end, :]
            Z[i,h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[:, :, :, c])

    

    # Making sure your output shape is correct
    assert(Z.shape == (m,n_H, n_W, n_C))
    

    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)

    return Z, cache

In [0]:
def relu(X):
  return np.maximum(X,0);

In [0]:
XX = relu(Z)
ZZ = np.reshape(Z,[Z.shape[1]*Z.shape[2]*Z.shape[3]])
WW = np.random.randn(1024,ZZ.shape[0])
M = np.dot(WW,ZZ);


WW2 = np.random.rand(100,M.shape[0])
M2 = np.dot(WW2,M)
M2.shape

In [0]:
def conv_backward(dZ, cache):
    
    
    (A_prev, W, b, hparameters) = cache
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    (m, n_H, n_W, n_C) = dZ.shape
    
    
    dA_prev = np.zeros((A_prev.shape))                           
    dW = np.zeros((W.shape))
    db = np.zeros((1,1,1,n_C))
    
    
    for i in range(m):                      
        
   
        a_prev = A_prev[i, :, :, :]  
        da_prev= dA_prev[i,:,:,:]
        
        for h in range(n_H):                   
            for w in range(n_W):               
                for c in range(n_C):           
                    
                    vert_start = stride*h
                    vert_end = f+vert_start
                    horiz_start =  stride*w
                    horiz_end = f+horiz_start
                    
                    a_slice = a_prev[vert_start:vert_end,horiz_start:horiz_end,:]

                    da_prev[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice*dZ[i,h,w,c]
                    db[:,:,:,c] += dZ[i,h,w,c]
                    
        dA_prev[i, :, :, :] = da_prev
    
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    print(dZ.shape)
    print(A_prev.shape)
    return dA_prev, dW, db